In [3]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# LangChain
- Open-source Framework
- Python, JS/TS
- Kombination von LLMs mit externen Funktionalitäten und Daten

## Model I/O

### Prompts
Die Anweisung/Frage/Aufgabestellung die das Modell bearbeiten soll.

### Language Models
Interfaces und Integrationen von verschiedenen Modellen
- LLMs: input text → output text (z.B. `text-davinci-003`)
- Chat models: input Liste von Chatnachrichten (System, Human, AI) → output Chatnachricht (z.B. `gpt-3.5-turbo`)

### Output Parsers
Wandeln die Textausgabe der Modelle in strukturierte Daten um.

In [52]:
from langchain.prompts import ChatPromptTemplate

system_template = ("You are a helpful assistant who generates comma separated lists. "
                   "A user will pass in a category, and you should generate 5 objects in that category in a comma separated list. "
                   "ONLY return a comma separated list, and nothing more.")
human_template = "{text}"

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])
prompt_template.invoke({"text": "programming languages"})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant who generates comma separated lists. A user will pass in a category, and you should generate 5 objects in that category in a comma separated list. ONLY return a comma separated list, and nothing more.'), HumanMessage(content='programming languages')])

In [53]:
from langchain.chat_models import ChatOpenAI

# default: gpt-3.5-turbo
llm = ChatOpenAI()
llm.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?')

In [54]:
from langchain.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()

parser.invoke("python, typescript")

['python', 'typescript']

## Chains
Chains erlauben es mehrere Modelle (LLM, Chat) und Komponenten (Prompt, Parser, Retriever) zu verknüpfen.
Für das Erstellen von Chains wird vorrangig die [LangChain Expression Language (LCEL)](https://python.langchain.com/docs/expression_language/) verwendet.

```python
chain = prompt | model | parser
```

In [55]:
chain = prompt_template | llm | parser

chain.invoke({"text": "programming languages"})

['Python', 'Java', 'JavaScript', 'C++', 'Ruby']

## Memory
- Informationen aus vorausgegangenen Konversationen speichern
- Speichern und abrufen von Informationen (reading/writing)
- Mit Tokenlimit umgehen

In [56]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI()
memory = ConversationBufferMemory()
conversation_chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True    
)

In [45]:
conversation_chain.invoke("hi! My name is Dave")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hi! My name is Dave
AI:

> Finished chain.


{'input': 'hi! My name is Dave',
 'history': '',
 'response': "Hello Dave! It's a pleasure to meet you. How can I assist you today?"}

In [46]:
conversation_chain.invoke("What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi! My name is Dave
AI: Hello Dave! It's a pleasure to meet you. How can I assist you today?
Human: What is my name?
AI:

> Finished chain.


{'input': 'What is my name?',
 'history': "Human: hi! My name is Dave\nAI: Hello Dave! It's a pleasure to meet you. How can I assist you today?",
 'response': 'Your name is Dave.'}

In [47]:
conversation_chain.memory.chat_memory.messages

[HumanMessage(content='hi! My name is Dave'),
 AIMessage(content="Hello Dave! It's a pleasure to meet you. How can I assist you today?"),
 HumanMessage(content='What is my name?'),
 AIMessage(content='Your name is Dave.')]

## Retrieval
- Einbindung von Daten, die nicht Teil des Trainingssets des Modells sind
- Retrieval Augmented Generation (RAG) 
- Dokumente laden/transformieren, Embeddings, Vector Stores, Retrievers

In [57]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import FAISS

In [58]:
vectorstore = FAISS.from_texts(["Hacky Hour: Large Language Models for Developers "
                                "anwenden, integrieren und erweitern, "
                                "Degginger Regensburg, 14.11.2023, Einlass 18:00 Uhr, Start 18:30 Uhr"], embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)

In [23]:
chain.invoke("Wann und wo findet die Hacky Hour statt?")

'Die Hacky Hour findet am 14.11.2023 in Degginger Regensburg statt.'

In [59]:
chain.invoke("Um wie viel Uhr Startet die Hacky Hour?")

'Die Hacky Hour startet um 18:30 Uhr.'

## Agents
- Agents erlauben LLMs mit ihrer Umgebung zu interagieren (Funktionen, APIs).
- LLMs werden als Reasoning-Engine genutzt um zu entscheiden welche Aktionen ausgeführt werden sollen (Chains sind hardcoded)
- AgentAction: Welche Aktion soll ausgeführt werden (tool, tool_input)
- AgentFinish: Ist der Agent mit seiner Arbeit fertig und soll das Ergebnis zurückgeben
- intermediate_steps: Welche Schritte wurden bereits ausgeführt

In [6]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType, load_tools
from langchain.chat_models import ChatOpenAI
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

In [44]:
llm = ChatOpenAI(model="gpt-4-0613")

tools = load_tools(["serpapi", "llm-math"], llm=llm)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent_chain = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, 
    memory=memory,
    verbose=True
)

In [47]:
agent_chain.invoke("Who is leonardo di caprios current girlfriend and what is here age multiplied by 3?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Search",
    "action_input": "Leonardo DiCaprio's current girlfriend"
}
```
Observation: Leonardo DiCaprio and new girlfriend Vittoria Ceretti hit another Paris party.
Thought:```json
{
    "action": "Search",
    "action_input": "Vittoria Ceretti's age"
}
```
Observation: 25 years
Thought:```json
{
    "action": "Calculator",
    "action_input": "25 * 3"
}
```
Observation: Answer: 75
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Leonardo DiCaprio's current girlfriend is Vittoria Ceretti, and her age multiplied by 3 is 75."
}
```

> Finished chain.


{'input': 'Who is leonardo di caprios current girlfriend and what is here age multiplied by 3?',
 'chat_history': [HumanMessage(content='Who is leonardo di caprios current girlfriend and what is here age multiplied by 3?'),
  AIMessage(content="Leonardo DiCaprio's current girlfriend is Vittoria Ceretti, and her age multiplied by 3 is 75.")],
 'output': "Leonardo DiCaprio's current girlfriend is Vittoria Ceretti, and her age multiplied by 3 is 75."}

# OpenAI function with LangChain

In [ ]:
# TODO: openai function example with langchain